## Creating a model

In [1]:
from train import train_model
import os, glob
import numpy as np
import skimage.measure
import datagenerators
from keras.optimizers import Adam
import losses
import tensorflow as tf
import Network_Building
from keras.callbacks import ModelCheckpoint, TensorBoard
import keras.backend as K

Using TensorFlow backend.
c:\users\bmanderson\appdata\local\programs\python\python36\lib\site-packages\dicom\__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)


## Define base parameters

In [4]:
data_dir = os.path.join('..','Reg_Data')
model_dir = os.path.join('..','models')
gpu_id='0'
steps_per_epoch = 30
batch_size = 1
data_loss = 'mean_squared_error'

## Load in the atlas file

In [5]:
atlas_file='../reg_data/Atlas_Data.npy'
atlas_vol = np.load(atlas_file) #['vol'][np.newaxis, ..., np.newaxis]
z_max = 64
# atlas_vol = atlas_vol[:,:,factor:-factor,factor:-factor,:]
atlas_vol = skimage.measure.block_reduce(atlas_vol,(1, 2, 2, 2, 1), np.average)
if atlas_vol.shape[1] > z_max:
    atlas_vol = atlas_vol[:, -z_max:, ...]
holder = (1,z_max,256,256,1) - np.asarray(atlas_vol.shape)
val_differences = [[i,0] for i in holder]
atlas_vol = np.pad(atlas_vol, val_differences, 'constant', constant_values=(-1000))
vol_size = atlas_vol.shape[1:-1]

### Creating your model

This is the part that you have the most unique control over, what exactly will your model look like?

One good way of visualizing the model is to create it and look at the graphical architecture with tensorboard

Lets start off with something simple, a UNet with one concatentation layer

In [4]:
layers = {'Layer_0':{'Encoding':[16,32],'Decoding':[32,16,8]},
          'Base':{'Encoding':[64]}}
model_desc = 'Shallow_net' # Name of your model
# The numbers inside are the number of filter banks, you can have mulitple filter banks per layer

This should be imagined as a U net, the numbers within [] are the number of filters in the convolution blocks, the network will first perform convolutions downthe encoding side to the base, and then concatenate from the previous encoding layer before performing decoding convolutions

This might not look like much, but lets see what it looks like graphically

In [10]:
#!pip install tensorflow

You are using pip version 18.1, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [18]:
#%load_ext tensorboard.notebook

The tensorboard.notebook extension is already loaded. To reload it, use:
  %reload_ext tensorboard.notebook


## Create the model

In [5]:
K.clear_session()
model_class = Network_Building.new_model(image_size=vol_size,layers=layers, visualize=True)
model = model_class.model
tensorboard_output = os.path.join('..','Tensorboard_models',model_desc)
if not os.path.exists(tensorboard_output):
    os.makedirs(tensorboard_output)
print(os.path.abspath(tensorboard_output))
tensorboard = TensorBoard(log_dir=tensorboard_output, batch_size=2, write_graph=True, write_grads=False,
                          write_images=True, update_freq='epoch', histogram_freq=0)
tensorboard.set_model(model)
tensorboard._write_logs({},0)
model_output = os.path.join(model_dir, model_desc, 'Model_saves')
save_file_name = os.path.join(model_output,'weights-improvement-{epoch:02d}.hdf5')
checkpoint = ModelCheckpoint(save_file_name, save_weights_only=False, period=1)
callbacks = [checkpoint, tensorboard]

K:\Morfeus\AAPM_SummerSchool\voxelmorph\Tensorboard_models\Shallow_net


## Visualize model
Now go to the terminal location printed out above and type

tensorboard --logdir=. --host=localhost

## Load training data

In [8]:
train_vol_names = glob.glob(os.path.join(data_dir, '*Registered_Data.npy'))
train_example_gen = datagenerators.example_gen(train_vol_names, batch_size=batch_size)
atlas_vol_bs = np.repeat(atlas_vol, batch_size, axis=0)
data_gen = datagenerators.cvpr2018_gen(train_example_gen, atlas_vol_bs, batch_size=batch_size)

## Define hyperparameters
These are variables which you will manipulate in order to create the best model possible

In [6]:
learning_rate = 0.001 # Rate at which our gradients will change during each back propogation, typically in range of 1e-2 to 1e-5
number_of_epochs = 10 # The number of epochs to be trained, one epoch means that you have seen the entirety of your dataset
                      # However, since we defined steps per epoch this might not apply
regularization_parameter = 0.01 # Lambda in regularization equation
steps_per_epoch = 30
batch_normalization = True

In [ ]:
K.clear_session()
with K.tf.device('/cpu:0'):
    model_class = Network_Building.new_model(image_size=vol_size, layers=layers, batch_normalization=batch_normalization)
    model = model_class.model
    model.compile(optimizer=Adam(lr=learning_rate),
                     loss=[data_loss, losses.Grad('l2').loss],
                     loss_weights=[1.0, regularization_parameter])

    # fit
    model.fit_generator(data_gen,
                           initial_epoch=0,
                           epochs=number_of_epochs,
                           callbacks=callbacks,
                           steps_per_epoch=steps_per_epoch,
                           verbose=1)

Epoch 1/10
 1/30 [>.............................] - ETA: 12:47 - loss: 143269.8281 - spatial_transformer_1_loss: 143269.8281 - flow_loss: 1.2845e-08

In [75]:
K.clear_session()
with K.tf.device('/gpu:0'):
    model = Network_Building.new_model(image_size=vol_size, layers=layers, batch_normalization=batch_normalization)
    gpu_options = tf.GPUOptions(allow_growth=True)
    sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=False))
    K.set_session(sess)
    model.compile(optimizer=Adam(lr=learning_rate),
                     loss=[data_loss, losses.Grad('l2').loss],
                     loss_weights=[1.0, regularization_parameter])

    # fit
    model.fit_generator(data_gen,
                           initial_epoch=0,
                           epochs=number_of_epochs,
                           callbacks=callbacks,
                           steps_per_epoch=steps_per_epoch,
                           verbose=1)

Epoch 1/10
30/30 [==============================] - ETA: 10:40 - loss: 133980.7969 - spatial_transformer_2_loss: 133980.7969 - flow_loss: 9.6098e-0 - ETA: 5:31 - loss: 131502.8125 - spatial_transformer_2_loss: 131502.8125 - flow_loss: 8.2360e-0 - ETA: 3:48 - loss: 134196.9792 - spatial_transformer_2_loss: 134196.9792 - flow_loss: 3.1711e- - ETA: 2:54 - loss: 134063.4531 - spatial_transformer_2_loss: 134063.4531 - flow_loss: 7.6383e- - ETA: 2:22 - loss: 132965.6063 - spatial_transformer_2_loss: 132965.6063 - flow_loss: 0.0016   - ETA: 2:00 - loss: 136901.8620 - spatial_transformer_2_loss: 136901.8620 - flow_loss: 0.00 - ETA: 1:44 - loss: 135979.6864 - spatial_transformer_2_loss: 135979.6864 - flow_loss: 0.00 - ETA: 1:31 - loss: 134575.4990 - spatial_transformer_2_loss: 134575.4990 - flow_loss: 0.00 - ETA: 1:21 - loss: 133456.4201 - spatial_transformer_2_loss: 133456.4201 - flow_loss: 0.00 - ETA: 1:12 - loss: 132405.3617 - spatial_transformer_2_loss: 132405.3617 - flow_loss: 0.01 - ETA: 

Epoch 6/10
30/30 [==============================] - ETA: 46s - loss: 31754.5469 - spatial_transformer_2_loss: 31754.4688 - flow_loss: 7.906 - ETA: 45s - loss: 37667.7637 - spatial_transformer_2_loss: 37667.6836 - flow_loss: 8.083 - ETA: 43s - loss: 38709.8242 - spatial_transformer_2_loss: 38709.7409 - flow_loss: 8.326 - ETA: 41s - loss: 36963.3633 - spatial_transformer_2_loss: 36963.2773 - flow_loss: 8.567 - ETA: 40s - loss: 35990.0094 - spatial_transformer_2_loss: 35989.9207 - flow_loss: 8.838 - ETA: 38s - loss: 35104.2627 - spatial_transformer_2_loss: 35104.1719 - flow_loss: 9.053 - ETA: 36s - loss: 34043.6197 - spatial_transformer_2_loss: 34043.5271 - flow_loss: 9.233 - ETA: 35s - loss: 33256.5364 - spatial_transformer_2_loss: 33256.4426 - flow_loss: 9.359 - ETA: 33s - loss: 33161.9164 - spatial_transformer_2_loss: 33161.8218 - flow_loss: 9.448 - ETA: 32s - loss: 32514.8807 - spatial_transformer_2_loss: 32514.7854 - flow_loss: 9.528 - ETA: 30s - loss: 33312.1454 - spatial_transforme

Epoch 9/10
30/30 [==============================] - ETA: 46s - loss: 36217.3125 - spatial_transformer_2_loss: 36217.0859 - flow_loss: 22.67 - ETA: 44s - loss: 35092.0410 - spatial_transformer_2_loss: 35091.8105 - flow_loss: 23.09 - ETA: 43s - loss: 34830.0703 - spatial_transformer_2_loss: 34829.8333 - flow_loss: 23.72 - ETA: 41s - loss: 35144.1191 - spatial_transformer_2_loss: 35143.8799 - flow_loss: 23.92 - ETA: 40s - loss: 32847.8668 - spatial_transformer_2_loss: 32847.6285 - flow_loss: 23.81 - ETA: 38s - loss: 33061.0459 - spatial_transformer_2_loss: 33060.8102 - flow_loss: 23.57 - ETA: 36s - loss: 32919.9422 - spatial_transformer_2_loss: 32919.7081 - flow_loss: 23.40 - ETA: 35s - loss: 31632.9119 - spatial_transformer_2_loss: 31632.6782 - flow_loss: 23.36 - ETA: 33s - loss: 31728.3021 - spatial_transformer_2_loss: 31728.0688 - flow_loss: 23.34 - ETA: 32s - loss: 30847.8857 - spatial_transformer_2_loss: 30847.6523 - flow_loss: 23.35 - ETA: 30s - loss: 31042.5534 - spatial_transforme

Play around! Add more layers, convolutions, see if you can reduce your loss the most, change the bottom two files, make sure to rename your model

In [2]:
layers = {'Layer_0':{'Encoding':[16],'Decoding':[16,16]},
          'Layer_1':{'Encoding':[32],'Decoding':[32,32]},
          'Layer_2':{'Encoding':[32],'Decoding':[32]},
          'Layer_3':{'Encoding':[32],'Decoding':[32]},
          'Base':{'Encoding':[32]}}
model_desc = 'Deep_net' # Name of your model
# The numbers inside are the number of filter banks, you can have mulitple filter banks per layer

In [6]:
K.clear_session()
model_class = Network_Building.new_model(image_size=vol_size,layers=layers, visualize=True)
model = model_class.model
tensorboard_output = os.path.join('..','Tensorboard_models',model_desc)
if not os.path.exists(tensorboard_output):
    os.makedirs(tensorboard_output)
print(os.path.abspath(tensorboard_output))
tensorboard = TensorBoard(log_dir=tensorboard_output, batch_size=2, write_graph=True, write_grads=False,
                          write_images=True, update_freq='epoch', histogram_freq=0)
tensorboard.set_model(model)
tensorboard._write_logs({},0)
model_output = os.path.join(model_dir, model_desc, 'Model_saves')
save_file_name = os.path.join(model_output,'weights-improvement-{epoch:02d}.hdf5')
checkpoint = ModelCheckpoint(save_file_name, save_weights_only=False, period=1)
callbacks = [checkpoint, tensorboard]

K:\Morfeus\AAPM_SummerSchool\voxelmorph\Tensorboard_models\Deep_net
